In [13]:
from rdkit.Chem import AllChem
from scipy import stats
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from scipy import stats
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem.Scaffolds import MurckoScaffold
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef,confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns
import time
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

In [14]:
import pandas as pd
import numpy as np

In [32]:
assay_data = pd.read_csv("./assay_data_filtered_89_inchi.csv") 
assay_list= assay_data.columns.to_list()[1:-1]
assay_data = assay_data.drop(columns= ["smiles_r"])
assay_data

,InChICode_standardised,178_684,180_686,183_604,184_606,185_665,185_666,186_668,190_628,190_629,...,273_751,274_752,275_753,275_754,276_755,278_757,279_758,280_759,283_762,93_259
0,InChI=1S/C20H29NO5S/c1-13-5-3-4-6-14(2)9-18(22...,1.0,1.0,NaN,1.0,NaN,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,InChI=1S/C22H26N2O3/c1-25-20-9-5-18(6-10-20)4-...,1.0,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,InChI=1S/C10H12ClN5O3/c11-10-14-8(12)7-9(15-10...,1.0,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,InChI=1S/C31H39F3N4O5/c1-19-16-38(20(2)18-39)2...,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,InChI=1S/C36H40N4O4/c1-25-21-40(26(2)24-41)35(...,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15267,InChI=1S/C29H42O10/c1-15-22(32)23(33)24(34)25(...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
15268,InChI=1S/C9H11N3O4/c10-5-1-2-12-8-7(16-9(12)11...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
15269,InChI=1S/C22H22O8/c1-25-14-5-10(6-15(26-2)20(1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
15270,"InChI=1S/C40H71NO14/c1-15-27-40(11,48)33(44)22...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [33]:
len(assay_list)

89

In [35]:
CP_features = pd.read_csv("./CP_filtered_15272.csv") 
CP_features_list= CP_features.columns.to_list()[:-1]
#CP_features = CP_features.to_numpy()
CP_features

,Cells_Correlation_Correlation_RNA_AGP,Cells_Correlation_Costes_AGP_RNA,Cells_Correlation_K_AGP_DNA,Cells_Correlation_K_DNA_AGP,Cells_Correlation_K_DNA_ER,Cells_Correlation_K_DNA_Mito,Cells_Correlation_K_ER_DNA,Cells_Correlation_K_ER_Mito,Cells_Correlation_K_Mito_DNA,Cells_Correlation_K_RNA_DNA,...,Nuclei_Texture_SumVariance_AGP_10_0,Nuclei_Texture_SumVariance_DNA_10_0,Nuclei_Texture_SumVariance_ER_10_0,Nuclei_Texture_SumVariance_Mito_10_0,Nuclei_Texture_SumVariance_RNA_10_0,Nuclei_Texture_Variance_AGP_10_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_RNA_10_0,InChICode_standardised
0,0.213520,0.052182,0.340759,-0.107108,0.030104,0.043808,-0.058823,-0.076555,-0.080047,-0.060985,...,0.127041,0.300797,1.601798,0.649545,0.117986,-0.124655,0.875722,0.060316,-0.108497,InChI=1S/C20H29NO5S/c1-13-5-3-4-6-14(2)9-18(22...
1,0.500980,-0.049488,0.034006,0.009368,0.042325,0.289250,0.072184,0.236915,-0.090521,0.025497,...,0.113111,-0.031291,0.736375,-0.052299,-0.182771,0.020039,0.309519,0.146067,-0.135633,InChI=1S/C22H26N2O3/c1-25-20-9-5-18(6-10-20)4-...
2,0.554610,-0.037230,-0.144126,0.907236,0.762857,0.869041,-0.197808,0.050745,-0.265411,-0.223704,...,0.402068,-0.528586,0.894280,1.096441,0.201130,0.023909,0.209528,0.384470,0.242193,InChI=1S/C10H12ClN5O3/c11-10-14-8(12)7-9(15-10...
3,0.065095,0.052121,-0.021547,0.185901,0.081906,0.034104,-0.030566,-0.068275,-0.000007,-0.060455,...,-0.356829,-0.001533,0.040114,-0.145918,0.025249,-0.111571,0.086286,-0.083360,0.011623,InChI=1S/C31H39F3N4O5/c1-19-16-38(20(2)18-39)2...
4,0.116754,-0.089409,-0.006915,-0.116876,-0.017143,0.037999,0.008466,0.130419,-0.026270,0.015191,...,-0.565546,-0.050469,-0.191591,-0.484524,-0.230474,-0.250742,-0.028181,-0.188926,-0.048785,InChI=1S/C36H40N4O4/c1-25-21-40(26(2)24-41)35(...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15267,0.600421,0.006980,0.341992,-0.482678,-0.421343,0.160259,0.486249,0.404443,0.035852,0.378471,...,0.854714,0.141265,1.257099,0.942200,0.412648,1.143416,1.082932,0.892884,1.190745,InChI=1S/C29H42O10/c1-15-22(32)23(33)24(34)25(...
15268,0.357634,-0.102040,-0.155204,1.278826,0.829811,0.756359,-0.187745,-0.028950,-0.237447,-0.202268,...,-0.235418,-1.117448,-0.116413,0.036485,0.631407,-0.277122,-0.071896,-0.108458,0.260039,InChI=1S/C9H11N3O4/c10-5-1-2-12-8-7(16-9(12)11...
15269,0.385746,-0.010448,0.021540,-0.015133,-0.165300,0.102597,0.144109,0.218774,-0.041078,0.056075,...,0.822444,0.501358,0.956281,1.116960,0.726610,0.275630,0.493553,0.230773,0.175383,InChI=1S/C22H22O8/c1-25-14-5-10(6-15(26-2)20(1...
15270,0.078460,0.015044,-0.009505,-0.009737,-0.060708,-0.000053,-0.001857,0.068244,-0.021163,0.063623,...,-0.252019,0.261459,-0.101648,-0.197162,-0.011630,-0.169541,-0.096224,-0.132107,0.046411,"InChI=1S/C40H71NO14/c1-15-27-40(11,48)33(44)22..."


In [36]:
mfp_features = pd.read_csv("./Mfp_filtered_15272.csv") 
mfp_features_list = mfp_features.columns.to_list()[:-1]
mfp_features['fp'] = [(x) for x in mfp_features.iloc[:, :-1].to_numpy()]
mfp_features = mfp_features[["InChICode_standardised", "fp"]]
mfp_features

,InChICode_standardised,fp
0,InChI=1S/C20H29NO5S/c1-13-5-3-4-6-14(2)9-18(22...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,InChI=1S/C22H26N2O3/c1-25-20-9-5-18(6-10-20)4-...,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,InChI=1S/C10H12ClN5O3/c11-10-14-8(12)7-9(15-10...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,InChI=1S/C31H39F3N4O5/c1-19-16-38(20(2)18-39)2...,"[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ..."
4,InChI=1S/C36H40N4O4/c1-25-21-40(26(2)24-41)35(...,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
...,...,...
15267,InChI=1S/C29H42O10/c1-15-22(32)23(33)24(34)25(...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
15268,InChI=1S/C9H11N3O4/c10-5-1-2-12-8-7(16-9(12)11...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
15269,InChI=1S/C22H22O8/c1-25-14-5-10(6-15(26-2)20(1...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
15270,"InChI=1S/C40H71NO14/c1-15-27-40(11,48)33(44)22...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [37]:
broad_data= pd.merge(assay_data, CP_features)
broad_data= pd.merge(broad_data, mfp_features)
broad_data

,InChICode_standardised,178_684,180_686,183_604,184_606,185_665,185_666,186_668,190_628,190_629,...,Nuclei_Texture_SumVariance_AGP_10_0,Nuclei_Texture_SumVariance_DNA_10_0,Nuclei_Texture_SumVariance_ER_10_0,Nuclei_Texture_SumVariance_Mito_10_0,Nuclei_Texture_SumVariance_RNA_10_0,Nuclei_Texture_Variance_AGP_10_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_RNA_10_0,fp
0,InChI=1S/C20H29NO5S/c1-13-5-3-4-6-14(2)9-18(22...,1.0,1.0,NaN,1.0,NaN,NaN,1.0,1.0,1.0,...,0.127041,0.300797,1.601798,0.649545,0.117986,-0.124655,0.875722,0.060316,-0.108497,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,InChI=1S/C22H26N2O3/c1-25-20-9-5-18(6-10-20)4-...,1.0,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,...,0.113111,-0.031291,0.736375,-0.052299,-0.182771,0.020039,0.309519,0.146067,-0.135633,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,InChI=1S/C10H12ClN5O3/c11-10-14-8(12)7-9(15-10...,1.0,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,...,0.402068,-0.528586,0.894280,1.096441,0.201130,0.023909,0.209528,0.384470,0.242193,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,InChI=1S/C31H39F3N4O5/c1-19-16-38(20(2)18-39)2...,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,-0.356829,-0.001533,0.040114,-0.145918,0.025249,-0.111571,0.086286,-0.083360,0.011623,"[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ..."
4,InChI=1S/C36H40N4O4/c1-25-21-40(26(2)24-41)35(...,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,-0.565546,-0.050469,-0.191591,-0.484524,-0.230474,-0.250742,-0.028181,-0.188926,-0.048785,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15267,InChI=1S/C29H42O10/c1-15-22(32)23(33)24(34)25(...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.854714,0.141265,1.257099,0.942200,0.412648,1.143416,1.082932,0.892884,1.190745,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
15268,InChI=1S/C9H11N3O4/c10-5-1-2-12-8-7(16-9(12)11...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.235418,-1.117448,-0.116413,0.036485,0.631407,-0.277122,-0.071896,-0.108458,0.260039,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
15269,InChI=1S/C22H22O8/c1-25-14-5-10(6-15(26-2)20(1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.822444,0.501358,0.956281,1.116960,0.726610,0.275630,0.493553,0.230773,0.175383,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
15270,"InChI=1S/C40H71NO14/c1-15-27-40(11,48)33(44)22...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.252019,0.261459,-0.101648,-0.197162,-0.011630,-0.169541,-0.096224,-0.132107,0.046411,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [51]:
#Nested Cross validation
#stat_list_batch: contains assay number, mean, median, mode of tanimoto distance in that cycle, mcc and auc of that cycle
#detail_list_batch contains assay number, each inchi and its proba, pred, true values and ts
#pred_list_batch contains proba, pred and true values

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.under_sampling import RandomUnderSampler 

from collections import Counter
from numpy import argmax

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from scipy.stats import ks_2samp 
from sklearn.model_selection import cross_val_predict

def my_function_nested_rhs(assay, cycle):   
        
    detail_list_batch_train = []

    X_fp = df_assay_train.loc[:, "fp"].to_numpy()
    X_CP = df_assay_train[CP_features_list].to_numpy()
    
    #print(X)
    y = df_assay_train[assay].to_numpy()
    #print(y)
    
    
    #Split
    outercv = StratifiedKFold(n_splits=5,shuffle=True, random_state=cycle)
   
    for i, (train_index, test_index) in enumerate(outercv.split(X_fp, y)):
        
        train= df_assay_train.iloc[train_index]
        test= df_assay_train.iloc[test_index]
        #print(train)
        #print(test)
        
        
        #Prepare features
        train_x_fp, test_x_fp, train_y, test_y = list(train.fp), list(test.fp), train[assay], test[assay]
        col_list=[]
      
        train_x_cp=train[CP_features_list].to_numpy()
        test_x_cp=test[CP_features_list].to_numpy()
        
        #MFP
        print("Mfp_Model_Running for assay outercv#", i+1)    
        inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
         
        param_dist_grid = {
        'max_depth': [10,15,20],
        'max_features': randint(40, 100),
        'min_samples_leaf': randint(2, 15),
        'min_samples_split': randint(2, 15),
        'n_estimators':[100, 300, 400, 500],
        'bootstrap': [True, False],
        'oob_score': [False],
        'random_state': [42],
        'criterion': ['gini', 'entropy'],
        'n_jobs': [-1],
        'class_weight' : [None, 'balanced']
        }
        
        rf = RandomForestClassifier(n_jobs=-1)
        rsh = HalvingRandomSearchCV(estimator=rf, param_distributions=param_dist_grid, factor=2, random_state=42, n_jobs=40, verbose=0,  cv = inner_cv)

        rsh.fit(train_x_fp, train_y)

        print("Tuned Mode: ", rsh.best_params_)
        classifier = rsh.best_estimator_
        
        #classifier = RandomForestClassifier(n_estimators=300, n_jobs=-1)                      
        classifier.fit(train_x_fp, train_y)
        
        #Threshold Balancing
        cross_val_prob_fp = cross_val_predict(rsh.best_estimator_, train_x_fp, train_y, cv=inner_cv, method='predict_proba')[:, 1]
        # calculate roc curves
        fpr, tpr, thresholds = roc_curve(train_y, cross_val_prob_fp)
        # get the best threshold
        J = tpr - fpr
        ix = argmax(J)
        best_thresh_fp = thresholds[ix]
        print('Best Threshold=%f' % (best_thresh_fp))


        fp_proba = classifier.predict_proba(test_x_fp)[:,1]
        #fp_pred = classifier.predict(test_x_fp) 
        fp_pred  = [ 1 if x>best_thresh_fp  else 0 for x in fp_proba ] 
    
        #Cell Painting
        
        print("CP_Model_Running for assay outercv#", i+1)  
        inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
      
        rf = RandomForestClassifier(n_jobs=-1)
        rsh = HalvingRandomSearchCV(estimator=rf, param_distributions=param_dist_grid, factor=2, random_state=42, n_jobs=40, verbose=0,  cv = inner_cv)
        
        
        rsh.fit(train_x_cp, train_y)
        
        print("Tuned Mode: ", rsh.best_params_)
        classifier = rsh.best_estimator_    
        
        #classifier = RandomForestClassifier(n_estimators=300, n_jobs=-1)
        classifier.fit(train_x_cp, train_y)
        
        #Threshold Balancing
        cross_val_prob_cp = cross_val_predict(rsh.best_estimator_, train_x_cp, train_y, cv=inner_cv, method='predict_proba')[:, 1]
        # calculate roc curves
        fpr, tpr, thresholds = roc_curve(train_y, cross_val_prob_cp)
        # get the best threshold
        J = tpr - fpr
        ix = argmax(J)
        best_thresh_cp = thresholds[ix]
        print('Best Threshold=%f' % (best_thresh_cp))


        CP_proba = classifier.predict_proba(test_x_cp)[:,1]
        #CP_pred = classifier.predict(test_x_cp) 
        CP_pred  = [ 1 if x>best_thresh_cp  else 0 for x in CP_proba ] 
        
        fpr, tpr, _ = roc_curve(test_y, CP_proba)

        #combine results
        for InChICode_standardised, fp_proba, fp_pred, CP_proba, CP_pred, true_val, in zip(test.InChICode_standardised, fp_proba, fp_pred, CP_proba, CP_pred, test_y):
            detail_list_batch_train.append([assay, InChICode_standardised, fp_proba, fp_pred,best_thresh_fp, CP_proba, CP_pred, best_thresh_cp, true_val, "Training"])
    
    return detail_list_batch_train

In [52]:
#Predict External Test Set
#detail_list_batch contains assay number, each inchi and its proba, pred, true values and ts


def my_function_heldout_rhs(assay, cycle):   
        
        detail_list_batch_test = []

        train= df_assay_train
        test= df_assay_test

        
        #Prepare features
        train_x_fp, test_x_fp, train_y, test_y = list(train.fp), list(test.fp), train[assay], test[assay]
        col_list=[]
      
        train_x_cp=train[CP_features_list].to_numpy()
        test_x_cp=test[CP_features_list].to_numpy()
        
        #MFP
        print("Mfp_Model_Running for assay")    
        inner_cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
         
        param_dist_grid = {
        'max_depth': [10,15,20],
        'max_features': randint(40, 100),
        'min_samples_leaf': randint(2, 15),
        'min_samples_split': randint(2, 15),
        'n_estimators':[100, 300, 400, 500],
        'bootstrap': [True, False],
        'oob_score': [False],
        'random_state': [42],
        'criterion': ['gini', 'entropy'],
        'n_jobs': [-1],
        'class_weight' : [None, 'balanced']
        }
        
        rf = RandomForestClassifier(n_jobs=-1)
        rsh = HalvingRandomSearchCV(estimator=rf, param_distributions=param_dist_grid, factor=2, random_state=42, n_jobs=40, verbose=0,  cv = inner_cv)

        rsh.fit(train_x_fp, train_y)

        print("Tuned Mode: ", rsh.best_params_)
        classifier = rsh.best_estimator_
        
        #classifier = RandomForestClassifier(n_estimators=300, n_jobs=-1)                      
        classifier.fit(train_x_fp, train_y)
        
        #Threshold Balancing
        cross_val_prob_fp = cross_val_predict(rsh.best_estimator_, train_x_fp, train_y, cv=inner_cv, method='predict_proba')[:, 1]
        # calculate roc curves
        fpr, tpr, thresholds = roc_curve(train_y, cross_val_prob_fp)
        # get the best threshold
        J = tpr - fpr
        ix = argmax(J)
        best_thresh_fp = thresholds[ix]
        print('Best Threshold=%f' % (best_thresh_fp))


        fp_proba = classifier.predict_proba(test_x_fp)[:,1]
        #fp_pred = classifier.predict(test_x_fp) 
        fp_pred  = [ 1 if x>best_thresh_fp  else 0 for x in fp_proba ] 
    
        #Cell Painting
        
        print("CP_Model_Running for assay")  
        inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
      
        rf = RandomForestClassifier(n_jobs=-1)
        rsh = HalvingRandomSearchCV(estimator=rf, param_distributions=param_dist_grid, factor=2, random_state=42, n_jobs=40, verbose=0,  cv = inner_cv)
        
        
        rsh.fit(train_x_cp, train_y)
        
        print("Tuned Mode: ", rsh.best_params_)
        classifier = rsh.best_estimator_    
        
        #classifier = RandomForestClassifier(n_estimators=300, n_jobs=-1)
        classifier.fit(train_x_cp, train_y)
        
        #Threshold Balancing
        cross_val_prob_cp = cross_val_predict(rsh.best_estimator_, train_x_cp, train_y, cv=inner_cv, method='predict_proba')[:, 1]
        # calculate roc curves
        fpr, tpr, thresholds = roc_curve(train_y, cross_val_prob_cp)
        # get the best threshold
        J = tpr - fpr
        ix = argmax(J)
        best_thresh_cp = thresholds[ix]
        print('Best Threshold=%f' % (best_thresh_cp))


        CP_proba = classifier.predict_proba(test_x_cp)[:,1]
        #CP_pred = classifier.predict(test_x_cp) 
        CP_pred  = [ 1 if x>best_thresh_cp  else 0 for x in CP_proba ] 
        
        fpr, tpr, _ = roc_curve(test_y, CP_proba)

        #combine results
        for InChICode_standardised, fp_proba, fp_pred, CP_proba, CP_pred, true_val,  in zip(test.InChICode_standardised, fp_proba, fp_pred, CP_proba, CP_pred, test_y):
            detail_list_batch_test.append([assay, InChICode_standardised, fp_proba, fp_pred,best_thresh_fp, CP_proba, CP_pred, best_thresh_cp, true_val,  "HeldOut"])
    
        return detail_list_batch_test

In [53]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split

master_detail_list=[]
for assay in tqdm(assay_list[:]):
    
    print("Assay ", assay )
    df_assay= broad_data[pd.notnull(broad_data[assay])]

    print("Total Compounds ", len(df_assay) )
    inactives_n=len(df_assay[df_assay[assay] == 0])
    print("Inactive Compounds ", inactives_n)
    actives_n=len(df_assay[df_assay[assay] == 1])
    print("Active Compounds ", actives_n)
    
    df_assay_train, df_assay_test= train_test_split(df_assay, stratify=df_assay[[assay]], test_size=0.20, random_state=42)
    
    print("Perform nested cross validation on training set")
    detail_list_train =  my_function_nested_rhs(assay, 42)
   
    
    print("Predictions on held out set")
    detail_list_test = my_function_heldout_rhs(assay, 42)

    
    master_detail_list.append(detail_list_train) 
    master_detail_list.append(detail_list_test) 
    
    

  0%|                                                    | 0/89 [00:00<?, ?it/s]

Assay  178_684
Total Compounds  272
Inactive Compounds  204
Active Compounds  68
Perform nested cross validation on training set
Mfp_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': False, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 20, 'max_features': 76, 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 100, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.481794
CP_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 10, 'max_features': 99, 'min_samples_leaf': 3, 'min_samples_split': 11, 'n_estimators': 100, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.269814
Mfp_Model_Running for assay outercv# 2
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 10, 'max_features': 99, 'min_samples_leaf': 3, 'min_samples_split': 11, 'n_estimators': 100, 'n_jobs': -1, 'oob_score': False, 'random

  1%|▍                                         | 1/89 [01:08<1:40:01, 68.20s/it]

Best Threshold=0.306156
Assay  180_686
Total Compounds  204
Inactive Compounds  153
Active Compounds  51
Perform nested cross validation on training set
Mfp_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': False, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 10, 'max_features': 88, 'min_samples_leaf': 12, 'min_samples_split': 12, 'n_estimators': 400, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.478389
CP_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': False, 'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 20, 'max_features': 86, 'min_samples_leaf': 4, 'min_samples_split': 13, 'n_estimators': 400, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.190705
Mfp_Model_Running for assay outercv# 2
Tuned Mode:  {'bootstrap': False, 'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 20, 'max_features': 86, 'min_samples_leaf': 4, 'min_samples_split': 13, 'n_estimators': 400, '

  2%|▉                                         | 2/89 [02:19<1:41:45, 70.18s/it]

...

Best Threshold=0.328330
Assay  283_762
Total Compounds  428
Inactive Compounds  321
Active Compounds  107
Perform nested cross validation on training set
Mfp_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': False, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 20, 'max_features': 99, 'min_samples_leaf': 8, 'min_samples_split': 13, 'n_estimators': 100, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.196414
CP_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 15, 'max_features': 83, 'min_samples_leaf': 3, 'min_samples_split': 11, 'n_estimators': 300, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.446607
Mfp_Model_Running for assay outercv# 2
Tuned Mode:  {'bootstrap': False, 'class_weight': None, 'criterion': 'gini', 'max_depth': 15, 'max_features': 56, 'min_samples_leaf': 5, 'min_samples_split': 3, 'n_estimators': 500, 'n_jobs': -1, 'oob_sc

 99%|███████████████████████████████████████▌| 88/89 [3:34:17<01:56, 116.73s/it]

Best Threshold=0.388344
Assay  93_259
Total Compounds  127
Inactive Compounds  93
Active Compounds  34
Perform nested cross validation on training set
Mfp_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': True, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 20, 'max_features': 50, 'min_samples_leaf': 12, 'min_samples_split': 9, 'n_estimators': 100, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.482580
CP_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': False, 'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 15, 'max_features': 99, 'min_samples_leaf': 6, 'min_samples_split': 2, 'n_estimators': 500, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.141277
Mfp_Model_Running for assay outercv# 2
Tuned Mode:  {'bootstrap': False, 'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 20, 'max_features': 61, 'min_samples_leaf': 6, 'min_samples_split': 3, 'n_estimators': 500, 'n_jobs

100%|████████████████████████████████████████| 89/89 [3:35:41<00:00, 145.41s/it]

Best Threshold=0.428485


In [54]:
flatList_master_detail_list = [ item for elem in master_detail_list for item in elem]
list_of_lists_df = pd.DataFrame(flatList_master_detail_list,columns=["assay","InChICode_standardised","fp_proba", "fp_pred", "fp_threshold", "CP_proba", 
                                                     "CP_pred","CP_threshold","true", "Data"])
list_of_lists_df["MFP_Correct"] = list_of_lists_df["fp_pred"]==list_of_lists_df["true"]
list_of_lists_df["CP_Correct"] = list_of_lists_df["CP_pred"]==list_of_lists_df["true"]
list_of_lists_df

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,Data,MFP_Correct,CP_Correct
0,178_684,InChI=1S/C46H56N4O10/c1-7-42(55)22-28-23-45(40...,0.853763,1,0.481794,0.893879,1,0.269814,1.0,Training,True,True
1,178_684,InChI=1S/C30H42ClN3O5S/c1-21-17-34(22(2)20-35)...,0.357098,0,0.481794,0.412432,1,0.269814,1.0,Training,False,True
2,178_684,InChI=1S/C31H44N4O6/c1-22-18-35(23(2)21-36)31(...,0.046336,0,0.481794,0.167905,0,0.269814,0.0,Training,True,True
3,178_684,InChI=1S/C35H60N4O5/c1-26-22-39(27(2)25-40)35(...,0.039431,0,0.481794,0.182939,0,0.269814,0.0,Training,True,True
4,178_684,InChI=1S/C29H39N5O5/c1-19-16-34(20(2)18-35)28(...,0.345300,0,0.481794,0.091189,0,0.269814,0.0,Training,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...
58482,93_259,InChI=1S/C12H11N3O2S/c1-8-7-9(2)15(13-8)12-10-...,0.127744,0,0.245120,0.513325,1,0.428485,0.0,HeldOut,True,False
58483,93_259,InChI=1S/C17H16N2O4S/c1-22-14-7-8-16(23-2)17(1...,0.505973,1,0.245120,0.322499,0,0.428485,0.0,HeldOut,False,True
58484,93_259,InChI=1S/C24H21N3O2S/c28-23-19-8-4-5-9-21(19)3...,0.363862,1,0.245120,0.359675,0,0.428485,1.0,HeldOut,True,False
58485,93_259,InChI=1S/C18H22N2O4S/c1-14(2)15-5-7-16(8-6-15)...,0.290175,1,0.245120,0.260526,0,0.428485,0.0,HeldOut,False,True


In [55]:
len(list_of_lists_df[list_of_lists_df["Data"]=="Training"])

46755

In [56]:
len(list_of_lists_df[list_of_lists_df["Data"]=="HeldOut"])

11732

In [57]:
#Probability balancing to thresholds

new_results= pd.DataFrame()
new_new_results = pd.DataFrame()

for t in list(list_of_lists_df["fp_threshold"].unique()):
        
        detail_list_fold=list_of_lists_df[list_of_lists_df["fp_threshold"]==t]
        print(len(detail_list_fold["fp_proba"].values), "Compounds in Fold with Threshold: ", np.round(t, 3))
        #print(detail_list_fold["fp_proba"].values)
        k_list=[]
        for p in detail_list_fold["fp_proba"].values:
            if p<=t:
                #print(p)
                k=p*0.5/t

            else:
                #print(p)
                k=1-((1-p)*0.5)/(1-t)

            k_list.append(k)
        #print(k_list)
        detail_list_fold["fp_proba_scaled"]=k_list
        new_results=pd.concat([new_results, detail_list_fold])
       
for t in list(new_results["CP_threshold"].unique()):
        
        detail_list_fold=new_results[new_results["CP_threshold"]==t]
        print(len(detail_list_fold["fp_proba"].values), "Compounds in Fold with Threshold: ", np.round(t, 3))
        #print(detail_list_fold["fp_proba"].values)
        k_list=[]
        for p in detail_list_fold["CP_proba"].values:
            if p<=t:
                #print(p)
                k=p*0.5/t

            else:
                #print(p)
                k=1-((1-p)*0.5)/(1-t)

            k_list.append(k)
        #print(k_list)
        detail_list_fold["CP_proba_scaled"]=k_list    
        
        new_new_results=pd.concat([new_new_results, detail_list_fold])

new_new_results

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


44 Compounds in Fold with Threshold:  0.482
44 Compounds in Fold with Threshold:  0.299
43 Compounds in Fold with Threshold:  0.417
43 Compounds in Fold with Threshold:  0.555
43 Compounds in Fold with Threshold:  0.361
55 Compounds in Fold with Threshold:  0.421
33 Compounds in Fold with Threshold:  0.478
33 Compounds in Fold with Threshold:  0.54
33 Compounds in Fold with Threshold:  0.655
32 Compounds in Fold with Threshold:  0.625
32 Compounds in Fold with Threshold:  0.71
41 Compounds in Fold with Threshold:  0.816
29 Compounds in Fold with Threshold:  0.492
29 Compounds in Fold with Threshold:  0.557
29 Compounds in Fold with Threshold:  0.771
29 Compounds in Fold with Threshold:  0.509
28 Compounds in Fold with Threshold:  0.454
36 Compounds in Fold with Threshold:  0.389
50 Compounds in Fold with Threshold:  0.447
50 Compounds in Fold with Threshold:  0.417
50 Compounds in Fold with Threshold:  0.258
50 Compounds in Fold with Threshold:  0.207
49 Compounds in Fold with Threshol

37 Compounds in Fold with Threshold:  0.469
37 Compounds in Fold with Threshold:  0.395
47 Compounds in Fold with Threshold:  0.327
204 Compounds in Fold with Threshold:  0.204
204 Compounds in Fold with Threshold:  0.279
204 Compounds in Fold with Threshold:  0.321
204 Compounds in Fold with Threshold:  0.275
204 Compounds in Fold with Threshold:  0.132
256 Compounds in Fold with Threshold:  0.234
27 Compounds in Fold with Threshold:  0.309
26 Compounds in Fold with Threshold:  0.536
26 Compounds in Fold with Threshold:  0.628
26 Compounds in Fold with Threshold:  0.56
26 Compounds in Fold with Threshold:  0.507
33 Compounds in Fold with Threshold:  0.286
180 Compounds in Fold with Threshold:  0.438
179 Compounds in Fold with Threshold:  0.267
179 Compounds in Fold with Threshold:  0.272
179 Compounds in Fold with Threshold:  0.255
179 Compounds in Fold with Threshold:  0.285
224 Compounds in Fold with Threshold:  0.419
98 Compounds in Fold with Threshold:  0.242
98 Compounds in Fold 

152 Compounds in Fold with Threshold:  0.275
152 Compounds in Fold with Threshold:  0.176
152 Compounds in Fold with Threshold:  0.272
151 Compounds in Fold with Threshold:  0.234
151 Compounds in Fold with Threshold:  0.222
190 Compounds in Fold with Threshold:  0.217
50 Compounds in Fold with Threshold:  0.223
49 Compounds in Fold with Threshold:  0.08
49 Compounds in Fold with Threshold:  0.311
49 Compounds in Fold with Threshold:  0.28
49 Compounds in Fold with Threshold:  0.338
62 Compounds in Fold with Threshold:  0.457
29 Compounds in Fold with Threshold:  0.279
29 Compounds in Fold with Threshold:  0.491
29 Compounds in Fold with Threshold:  0.381
29 Compounds in Fold with Threshold:  0.453
28 Compounds in Fold with Threshold:  0.465
36 Compounds in Fold with Threshold:  0.164
69 Compounds in Fold with Threshold:  0.52
69 Compounds in Fold with Threshold:  0.511
68 Compounds in Fold with Threshold:  0.213
68 Compounds in Fold with Threshold:  0.274
68 Compounds in Fold with Thr

37 Compounds in Fold with Threshold:  0.323
37 Compounds in Fold with Threshold:  0.416
37 Compounds in Fold with Threshold:  0.43
37 Compounds in Fold with Threshold:  0.324
37 Compounds in Fold with Threshold:  0.504
47 Compounds in Fold with Threshold:  0.256
28 Compounds in Fold with Threshold:  0.353
28 Compounds in Fold with Threshold:  0.408
27 Compounds in Fold with Threshold:  0.236
27 Compounds in Fold with Threshold:  0.467
27 Compounds in Fold with Threshold:  0.519
35 Compounds in Fold with Threshold:  0.315
148 Compounds in Fold with Threshold:  0.383
147 Compounds in Fold with Threshold:  0.388
147 Compounds in Fold with Threshold:  0.293
147 Compounds in Fold with Threshold:  0.257
147 Compounds in Fold with Threshold:  0.363
184 Compounds in Fold with Threshold:  0.323
23 Compounds in Fold with Threshold:  0.356
23 Compounds in Fold with Threshold:  0.321
23 Compounds in Fold with Threshold:  0.361
23 Compounds in Fold with Threshold:  0.166
23 Compounds in Fold with T

187 Compounds in Fold with Threshold:  0.234
187 Compounds in Fold with Threshold:  0.27
235 Compounds in Fold with Threshold:  0.26
103 Compounds in Fold with Threshold:  0.312
103 Compounds in Fold with Threshold:  0.293
103 Compounds in Fold with Threshold:  0.296
103 Compounds in Fold with Threshold:  0.273
103 Compounds in Fold with Threshold:  0.266
129 Compounds in Fold with Threshold:  0.266
50 Compounds in Fold with Threshold:  0.22
50 Compounds in Fold with Threshold:  0.244
50 Compounds in Fold with Threshold:  0.385
50 Compounds in Fold with Threshold:  0.292
49 Compounds in Fold with Threshold:  0.264
63 Compounds in Fold with Threshold:  0.213
82 Compounds in Fold with Threshold:  0.263
81 Compounds in Fold with Threshold:  0.443
81 Compounds in Fold with Threshold:  0.337
81 Compounds in Fold with Threshold:  0.201
81 Compounds in Fold with Threshold:  0.259
102 Compounds in Fold with Threshold:  0.309
60 Compounds in Fold with Threshold:  0.305
60 Compounds in Fold with

20 Compounds in Fold with Threshold:  0.042
26 Compounds in Fold with Threshold:  0.428


,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,Data,MFP_Correct,CP_Correct,fp_proba_scaled,CP_proba_scaled
0,178_684,InChI=1S/C46H56N4O10/c1-7-42(55)22-28-23-45(40...,0.853763,1,0.481794,0.893879,1,0.269814,1.0,Training,True,True,0.858900,0.927333
1,178_684,InChI=1S/C30H42ClN3O5S/c1-21-17-34(22(2)20-35)...,0.357098,0,0.481794,0.412432,1,0.269814,1.0,Training,False,True,0.370592,0.597659
2,178_684,InChI=1S/C31H44N4O6/c1-22-18-35(23(2)21-36)31(...,0.046336,0,0.481794,0.167905,0,0.269814,0.0,Training,True,True,0.048086,0.311150
3,178_684,InChI=1S/C35H60N4O5/c1-26-22-39(27(2)25-40)35(...,0.039431,0,0.481794,0.182939,0,0.269814,0.0,Training,True,True,0.040921,0.339009
4,178_684,InChI=1S/C29H39N5O5/c1-19-16-34(20(2)18-35)28(...,0.345300,0,0.481794,0.091189,0,0.269814,0.0,Training,True,True,0.358348,0.168984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58482,93_259,InChI=1S/C12H11N3O2S/c1-8-7-9(2)15(13-8)12-10-...,0.127744,0,0.245120,0.513325,1,0.428485,0.0,HeldOut,True,False,0.260575,0.574224
58483,93_259,InChI=1S/C17H16N2O4S/c1-22-14-7-8-16(23-2)17(1...,0.505973,1,0.245120,0.322499,0,0.428485,0.0,HeldOut,False,True,0.672778,0.376325
58484,93_259,InChI=1S/C24H21N3O2S/c28-23-19-8-4-5-9-21(19)3...,0.363862,1,0.245120,0.359675,0,0.428485,1.0,HeldOut,True,False,0.578650,0.419705
58485,93_259,InChI=1S/C18H22N2O4S/c1-14(2)15-5-7-16(8-6-15)...,0.290175,1,0.245120,0.260526,0,0.428485,0.0,HeldOut,False,True,0.529843,0.304008


In [58]:
new_new_results.to_csv("Predictions_train_heldout_scaled_prob_all_assays.csv", index=False)